In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.utils import resample
from sklearn.feature_selection import SelectKBest, f_classif
%matplotlib inline
import statsmodels.api as sm

In [25]:
df = pd.read_csv('C:\\Users\\Square Bear\\Documents\\Thinkful\\Datasets\\2008.csv.bz2')
pd.options.display.max_columns = None
df.head(10)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,N612SW,88.0,90.0,78.0,-6.0,-4.0,IND,BWI,515,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0
5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,378,N726SW,101.0,115.0,87.0,11.0,25.0,IND,JAX,688,4.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
6,2008,1,3,4,1937.0,1830,2037.0,1940,WN,509,N763SW,240.0,250.0,230.0,57.0,67.0,IND,LAS,1591,3.0,7.0,0,NaN,0,10.0,0.0,0.0,0.0,47.0
7,2008,1,3,4,1039.0,1040,1132.0,1150,WN,535,N428WN,233.0,250.0,219.0,-18.0,-1.0,IND,LAS,1591,7.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
8,2008,1,3,4,617.0,615,652.0,650,WN,11,N689SW,95.0,95.0,70.0,2.0,2.0,IND,MCI,451,6.0,19.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
9,2008,1,3,4,1620.0,1620,1639.0,1655,WN,810,N648SW,79.0,95.0,70.0,-16.0,0.0,IND,MCI,451,3.0,6.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN


Name 	            Description
1 	Year 	            1987-2008
2 	Month 	            1-12
3 	DayofMonth 	        1-31
4 	DayOfWeek 	        1 (Monday) - 7 (Sunday)
5 	DepTime 	        actual departure time (local, hhmm)
6 	CRSDepTime 	        scheduled departure time (local, hhmm)
7 	ArrTime 	        actual arrival time (local, hhmm)
8 	CRSArrTime 	        scheduled arrival time (local, hhmm)
9 	UniqueCarrier 	    unique carrier code
10 	FlightNum 	        flight number
11 	TailNum 	        plane tail number
12 	ActualElapsedTime 	in minutes
13 	CRSElapsedTime 	    in minutes
14 	AirTime 	        in minutes
15 	ArrDelay 	        arrival delay, in minutes
16 	DepDelay 	        departure delay, in minutes
17 	Origin 	            origin IATA airport code
18 	Dest 	            destination IATA airport code
19 	Distance 	        in miles
20 	TaxiIn 	            taxi in time, in minutes
21 	TaxiOut 	        taxi out time in minutes
22 	Cancelled 	        was the flight cancelled?
23 	CancellationCode 	reason for cancellation (A = carrier, B = weather, C = NAS, D = security)
24 	Diverted 	        1 = yes, 0 = no
25 	CarrierDelay 	    in minutes
26 	WeatherDelay 	    in minutes
27 	NASDelay 	        in minutes
28 	SecurityDelay 	    in minutes
29 	LateAircraftDelay 	in minutes

In [12]:
df['DayOfWeek'].value_counts()

3    1039665
1    1036201
5    1035166
4    1032224
2    1032049
7     976887
6     857536
Name: DayOfWeek, dtype: int64

In [22]:
df['DepTime'].value_counts()

555.0     17227
600.0     16436
700.0     15169
655.0     15123
557.0     14686
556.0     14563
558.0     14071
559.0     12566
658.0     11849
657.0     11849
656.0     11834
755.0     11419
825.0     11305
659.0     11222
630.0     11190
900.0     11183
855.0     11066
1700.0    11006
1655.0    10736
625.0     10728
1500.0    10581
1100.0    10572
1120.0    10470
1000.0    10428
554.0     10425
830.0     10400
1125.0    10368
1325.0    10245
800.0     10125
1330.0    10099
          ...  
414.0         8
242.0         8
418.0         8
352.0         8
413.0         8
416.0         7
348.0         7
425.0         7
349.0         7
422.0         7
357.0         7
308.0         7
343.0         7
253.0         7
254.0         7
337.0         7
353.0         6
421.0         6
312.0         6
341.0         6
336.0         5
420.0         5
429.0         5
302.0         5
338.0         4
344.0         4
342.0         4
346.0         3
347.0         3
417.0         3
Name: DepTime, Length: 1

In [23]:
df['ArrTime'].value_counts()

1020.0    8435
1400.0    8365
1015.0    8289
1410.0    8286
1610.0    8254
1405.0    8228
1010.0    8206
1025.0    8184
1210.0    8177
1800.0    8173
1810.0    8172
1620.0    8165
1215.0    8160
1625.0    8147
1615.0    8121
1220.0    8111
1600.0    8108
1630.0    8104
1540.0    8066
1030.0    8053
1415.0    8049
1745.0    8031
1225.0    8028
1640.0    8024
1815.0    8022
1755.0    8019
1205.0    8015
1350.0    8009
1830.0    7996
1555.0    7994
          ... 
355.0       45
403.0       45
349.0       45
412.0       44
404.0       44
418.0       43
409.0       41
348.0       41
406.0       41
353.0       40
345.0       40
334.0       39
341.0       39
419.0       39
402.0       39
347.0       38
343.0       38
354.0       38
408.0       38
407.0       38
346.0       36
405.0       36
359.0       35
358.0       35
401.0       34
352.0       34
356.0       34
414.0       33
416.0       31
411.0       31
Name: ArrTime, Length: 1440, dtype: int64

In [24]:
df['UniqueCarrier'].value_counts()

WN    1201754
AA     604885
OO     567159
MQ     490693
US     453589
DL     451931
UA     449515
XE     374510
NW     347652
CO     298455
EV     280575
9E     262208
FL     261684
YV     254930
OH     197607
B6     196091
AS     151102
F9      95762
HA      61826
AQ       7800
Name: UniqueCarrier, dtype: int64

In [26]:
df.fillna(0)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,0,0,0.0,0.0,0.0,0.0,0.0
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,0,0,0.0,0.0,0.0,0.0,0.0
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,N612SW,88.0,90.0,78.0,-6.0,-4.0,IND,BWI,515,3.0,7.0,0,0,0,0.0,0.0,0.0,0.0,0.0
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,0,0,2.0,0.0,0.0,0.0,32.0
5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,378,N726SW,101.0,115.0,87.0,11.0,25.0,IND,JAX,688,4.0,10.0,0,0,0,0.0,0.0,0.0,0.0,0.0
6,2008,1,3,4,1937.0,1830,2037.0,1940,WN,509,N763SW,240.0,250.0,230.0,57.0,67.0,IND,LAS,1591,3.0,7.0,0,0,0,10.0,0.0,0.0,0.0,47.0
7,2008,1,3,4,1039.0,1040,1132.0,1150,WN,535,N428WN,233.0,250.0,219.0,-18.0,-1.0,IND,LAS,1591,7.0,7.0,0,0,0,0.0,0.0,0.0,0.0,0.0
8,2008,1,3,4,617.0,615,652.0,650,WN,11,N689SW,95.0,95.0,70.0,2.0,2.0,IND,MCI,451,6.0,19.0,0,0,0,0.0,0.0,0.0,0.0,0.0
9,2008,1,3,4,1620.0,1620,1639.0,1655,WN,810,N648SW,79.0,95.0,70.0,-16.0,0.0,IND,MCI,451,3.0,6.0,0,0,0,0.0,0.0,0.0,0.0,0.0
